In [1]:
import pandas as pd
import numpy as np

import glob, os
from os.path import isfile
done_files=[]
os.chdir("/data/dharp/compounds/datasets/stats/")
cur_dir='/data/dharp/compounds/datasets/stats/'
for file in glob.glob("*.txt"):
    done_files.append(file.split('.')[0])

In [42]:
df_list=[pd.read_csv(cur_dir+cur_file+'.txt',sep="\t",header=None) for cur_file in done_files]

In [7]:
tt=pd.read_csv('/data/dharp/compounds/datasets/compounds.csv',sep='\t')
tt

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [6]:
tt.groupby(['modifier','head'])['count'].sum().sort_values().tail(40)

modifier    head       
common      prayer          470397
capital     stock           472644
catholic    university      476813
central     committee       483908
chief       justice         503431
american    revolution      521674
great       britain         536210
air         force           546698
central     government      571291
balance     sheet           574158
american    society         607238
dining      room            608095
east        coast           614320
dorsal      surface         617137
jesus       christ          627987
indian      affairs         628365
black       sea             645886
cultural    revolution      670382
dialog      box             689594
end         result          690635
common      pleas           693589
cold        war             706250
american    association     721372
british     isles           724410
            empire          918169
cross       section         958416
death       penalty         967603
attorney    general         989

In [2]:
df_path='/data/dharp/compounds/datasets/entire_df/'
df_files=[]
for filename in glob.glob(df_path+'*parq'):
    #print(filename)
    df_files.append(filename)

In [3]:
def ner_lemma_reducer(sent):
    ner_cat=None
    lemma=[]
    parsed_sent=nlp(sent)
    for token in parsed_sent:
        lemma.append(token.lemma_)
    lemma_sent=' '.join(lemma)
    
    if parsed_sent.ents:
        for e in parsed_sent.ents:
            #print(e.text,e.label_)
            if e.label_=='PERSON':
                ner_cat='PERSON'
    else:
        ner_cat=None
    #print(sent,ner_cat,lemma_sent)
    return pd.Series([ner_cat,lemma_sent])

In [4]:
br_to_us=pd.read_csv('data/spelling.csv',sep='\t')
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))


spelling_replacement={'context':br_to_us_dict,'modifier':br_to_us_dict,'head':br_to_us_dict,'word':br_to_us_dict}


any_word=r'.+_.+'
any_noun=r'.+_noun'
proper_noun=r'[a-z.-]+_noun'
space=r'\s'

In [5]:
ff=fastparquet.ParquetFile('/data/dharp/compounds/datasets/entire_df/df_4.parq')
current_df = next(iter(ff.iter_row_groups()))

In [6]:
len(ff.row_groups)

60

In [7]:
current_df.year=current_df.year.astype('int32')
df=current_df.head(100000).copy()
df=df.groupby(['fivegram_pos','year']).sum('count')

In [10]:
new_df=pd.DataFrame(df.index.get_level_values('fivegram_pos').unique().tolist())
new_df.columns=['old_index']
new_df[['l1_pos','word_pos','r1_pos','r2_pos','r3_pos']]=new_df['old_index'].str.split(space,expand=True,n=5)
new_df['sent']=new_df.l1_pos.str.split('_').str[0]+" "+new_df.word_pos.str.split('_').str[0]+" "+new_df.r1_pos.str.split('_').str[0]+" "+new_df.r2_pos.str.split('_').str[0]+" "+new_df.r3_pos.str.split('_').str[0]
new_df['pos']=new_df.l1_pos.str.split('_').str[1]+" "+new_df.word_pos.str.split('_').str[1]+" "+new_df.r1_pos.str.split('_').str[1]+" "+new_df.r2_pos.str.split('_').str[1]+" "+new_df.r3_pos.str.split('_').str[1]
#new_df=new_df[['old_index','sent','pos']]
new_df[['IS_PERSON','new_sent']]=new_df['sent'].apply(ner_lemma_reducer)#,axis=1)
new_df

,old_index,l1_pos,word_pos,r1_pos,r2_pos,r3_pos,sent,pos,IS_PERSON,new_sent
0,he'th_adp a_det overtheer_noun and_conj makin_...,he'th_adp,a_det,overtheer_noun,and_conj,makin_verb,he'th a overtheer and makin,adp det noun conj verb,None,he'th a overtheer and makin
1,he_adj being_noun dead_pron yet_verb speaketh_...,he_adj,being_noun,dead_pron,yet_verb,speaketh_noun,he being dead yet speaketh,adj noun pron verb noun,None,-PRON- be dead yet speaketh
2,he_adj has_noun taken_verb off_verb in_adp,he_adj,has_noun,taken_verb,off_verb,in_adp,he has taken off in,adj noun verb verb adp,None,-PRON- have take off in
3,he_adj purpose_noun of_adp this_det book_noun,he_adj,purpose_noun,of_adp,this_det,book_noun,he purpose of this book,adj noun adp det noun,None,-PRON- purpose of this book
4,he_pron '_. knows_verb '_pron that_adp,he_pron,'_.,knows_verb,'_pron,that_adp,he ' knows ' that,pron . verb pron adp,None,-PRON- ' know ' that
...,...,...,...,...,...,...,...,...,...,...
2891,he_pron accepted_verb the_det office_noun of_adp,he_pron,accepted_verb,the_det,office_noun,of_adp,he accepted the office of,pron verb det noun adp,None,-PRON- accept the office of
2892,he_pron accepted_verb the_det position_noun in...,he_pron,accepted_verb,the_det,position_noun,in_adp,he accepted the position in,pron verb det noun adp,None,-PRON- accept the position in
2893,he_pron accepted_verb the_det post_noun at_adp,he_pron,accepted_verb,the_det,post_noun,at_adp,he accepted the post at,pron verb det noun adp,None,-PRON- accept the post at
2894,he_pron accepted_verb the_det post_noun with_adp,he_pron,accepted_verb,the_det,post_noun,with_adp,he accepted the post with,pron verb det noun adp,None,-PRON- accept the post with


In [9]:
new_df.l1_pos

AttributeError: 'DataFrame' object has no attribute 'l1_pos'

In [128]:
new_df.pos.str.replace('(?!noun)',repl='other',regex=True)

0       otheraotherdotherpother otherdothereothertothe...
1       otheraotherdotherjother notherootheruothernoth...
2       otheraotherdotherjother notherootheruothernoth...
3       otheraotherdotherjother notherootheruothernoth...
4       otherpotherrotheroothernother other.other othe...
                              ...                        
2891    otherpotherrotheroothernother othervothereothe...
2892    otherpotherrotheroothernother othervothereothe...
2893    otherpotherrotheroothernother othervothereothe...
2894    otherpotherrotheroothernother othervothereothe...
2895    otherpotherrotheroothernother othervothereothe...
Name: pos, Length: 2896, dtype: object

In [97]:
[^ ]

SyntaxError: invalid syntax (<ipython-input-97-7788f1f472b7>, line 1)

In [49]:
new_df.IS_PERSON.value_counts()

Series([], Name: IS_PERSON, dtype: int64)

In [14]:
doc=nlp(current_df.iloc[10].fivegram_pos)
for token in doc:
    print(token.text, token.pos_, token.dep_)

he'th_adp PROPN compound
a_det PROPN ROOT
overtheer_noun PROPN ROOT
and_conj PROPN compound
makin_verb PROPN ROOT
